In [1]:
import numpy as np
from mha021 import *

In [2]:
# Problem 1

# Define inputs
L = 2 # [m]
h = 2e-3 # [m]
r = 30e-3 # [m]
G = 70e9 # [Pa]
T0 = 300 # [Nm]
Ip = 2 * np.pi * r**3 * h # [m^4]

print('c)')
Le = L / 2

K = G * Ip / Le * np.array([
    [1, -1, 0],
    [-1, 2, -1],
    [0, -1, 1]
])\

f = np.zeros((3, 1))

f[1-1] = T0

bc_dofs = [3]
bc_vals = [0]

a, r = solve_eq(K, f, bc_dofs, bc_vals)
print(a)


print('d)')

s = 3e4 # [Nm]

K = G * Ip / Le * np.array([
    [1, -1, 0],
    [-1, 2, -1],
    [0, -1, 1]
])\
    +\
        np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, s]
])

f = np.zeros((3, 1))

f[1-1] = T0

a = np.linalg.solve(K, f)
print(a)

c)
[0.02526269 0.01263134 0.        ]
d)
[[0.03526269]
 [0.02263134]
 [0.01      ]]


In [3]:
# Problem 2

# Define inputs
T0 = 0 # [C]
k = 1 # [W / (m C)]
h = 1 # [W / m^3]
t = 1 # [m]
Lx = 1 # [m]
Ly = 1.25 # [m]
qn = 0 # on symmetry boundaries

import scipy.io
mesh = scipy.io.loadmat('mesh_data.mat') 

Coord = mesh['Coord']                         # [x, y] coords for each node
Dofs = mesh['Dofs']                           
Edof = mesh['Edof']                           # [element number, dof1, dof2, dof3]
Ex = mesh['Ex']                                
Ey = mesh['Ey']                               
right_dofs = mesh['right_dofs']                                                   
top_dofs = mesh['top_dofs']  
left_dofs = mesh['left_dofs']                                                   
bottom_dofs = mesh['bottom_dofs']  

print('d)')

D = k * np.eye(3)

num_nodes = mesh.nodes.shape[0]
num_el = mesh.elements.shape[0]
num_dofs = num_nodes

K = np.zeros((num_dofs, num_dofs))
f = np.zeros((num_dofs))

for el in range(num_el):
    nodes = mesh.nodes[mesh.elements[el] - 1]
    dofs = mesh.edofs[el, :]
    
    Ke, fe = flow2t_Ke_fe(nodes, t, D, Q=h)
    assem(K, Ke, dofs)
    assem(f, fe, dofs)

"""
def fe_edge(nodes, qn, t):
    x1, y1 = nodes[0]
    x2, y2 = nodes[1]
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) 
    fe_ed = t * qn * L / 2 * np.array([[1], [1]])
    return fe_ed
    
left_edges = mesh.edges['inner']
num_edges = len(left_edges)

for egde in range(len_edges):

    edge_nodes = left_edges[edge : edge + 2] - 1
    nodes = mesh.nodes[edge_nodes, :]
    dofs = mesh.edof[nodes + 1]
    
    fe = fe_edges(nodes, qn, t)
    assem(f, fe, dofs)
"""

bc_vals_right = np.zeros_like(right_dofs)
bc_vals_top = np.zeros_like(top_dofs)

bc_dofs = np.hstack([right_dofs, top_dofs])
bc_vals = np.hstack([bc_vals_right, bc_vals_top])

a, r = solve_eq(K, f, bc_dofs, bc_vals)

# Element 18
xe = Ex[el - 1, :]
ye = Ey[el - 1, :]

x = 0.650
y = 0.375

dofs_e = Edof[el - 1, :]
ae = a[dofs_e - 1]

# From formula sheet
Ae = ((xe[1] * ye[2] - xe[2] * ye[1]) - 
      (xe[0] * ye[2] - xe[2] * ye[0]) + 
      (xe[0] * ye[1] - xe[1] * ye[0])) / 2

# From formula sheet
Ne = np.array([
    xe[1] * ye[2] - xe[2] * ye[1] + (ye[1] - ye[2]) * x + (xe[2] - xe[1]) * y,
    xe[2] * ye[0] - xe[0] * ye[2] + (ye[2] - ye[0]) * x + (xe[2] - xe[0]) * y,
    xe[0] * ye[1] - xe[1] * ye[0] + (ye[0] - ye[1]) * x + (xe[1] - xe[0]) * y
])/(2 * Ae)

T_el18 = Ne @ ae

print(ae)


d)


AttributeError: 'dict' object has no attribute 'nodes'

In [ ]:
# Problem 3

# Define inputs
t = 0.015
E = 210e9
nu = 0.3

# Given vectors
vec1e = np.array([
    [0.010],
    [0.014]
])

vec2e = np.array([
    [0.021],
    [0.009]
])

vec3e = np.array([
    [0.015],
    [0.018]
])

In [ ]:

print('\na)')

# Assemble xe and ye vectors
xe = np.array([
    vec1e[0],
    vec2e[0],
    vec3e[0]
])

ye = np.array([
    vec1e[1],
    vec2e[1],
    vec3e[1]
])

xi, eta = sp.symbols('xi, eta')

# Shape functions
N1 = xi
N2 = eta
N3 = 1 - xi - eta

Ne_bar = sp.Matrix([[N1, N2, N3]])

Ne_bar_func = sp.lambdify((xi, eta), Ne_bar, 'numpy')

xi_val = 0.2
eta_val = 0.1

Ne_bar_val = Ne_bar_func(xi_val, eta_val)
print(f'Ne_bar_val: {Ne_bar_val}')
print(f'xe: {xe}')

# Compute global coordinates for point A
x_A = Ne_bar_val @ xe
y_A = Ne_bar_val @ ye

print(f'x_A: {x_A}')
print(f'y_A: {y_A}')



a)
Ne_bar_val: [[0.2 0.1 0.7]]
xe: [[0.01 ]
 [0.021]
 [0.015]]
x_A: [[0.0146]]
y_A: [[0.0163]]


In [ ]:
print('\nb)')

scale = 1e-5

# Given displacement vectors
u1e = scale * np.array([
    [4.1],
    [1.0]
])

u2e = scale * np.array([
    [3.5],
    [1.6]
])

u3e = scale * np.array([
    [3.8],
    [1.7]
])

# Assemble displacement vector
ae = np.vstack((u1e, u2e, u3e))
print(f'ae: {ae}')

Ne = sp.Matrix([
    [N1, 0, N2, 0, N3, 0],
    [0, N1, 0, N2, 0, N3]
])

Ne_func = sp.lambdify((xi, eta), Ne, 'numpy')

Ne_val = Ne_func(xi_val, eta_val)

ue = Ne_val @ ae
print(f'ue: {ue}')

x = Ne_bar @ xe
y = Ne_bar @ ye

J = sp.Matrix([x, y]).jacobian([xi, eta])

dNe_bar_dxi = Ne_bar.diff(xi)
dNe_bar_deta = Ne_bar.diff(eta)

# Lambdify expressions
J_func = sp.lambdify((xi, eta), J, 'numpy')
dNe_bar_dxi_func = sp.lambdify((xi, eta), dNe_bar_dxi, 'numpy')
dNe_bar_deta_func = sp.lambdify((xi, eta), dNe_bar_deta, 'numpy')

# Evaluate functions
J_val = J_func(xi_val, eta_val)
dNe_bar_dxi_val = dNe_bar_dxi_func(xi_val, eta_val)
dNe_bar_deta_val = dNe_bar_deta_func(xi_val, eta_val)

dNe_bar_dlocal = np.vstack([dNe_bar_dxi_val, dNe_bar_deta_val])

dNe_bar_dglobal = np.linalg.inv(J_val.T) @ dNe_bar_dlocal

Be = np.zeros((3, 6))
for i in range(3):
    id_even = 2 * i
    id_odd = 2 * i + 1

    Be[0, id_even] = dNe_bar_dglobal[0, i]
    Be[1, id_odd] = dNe_bar_dglobal[1, i]
    Be[2, id_odd] = dNe_bar_dglobal[0, i]
    Be[2, id_even] = dNe_bar_dglobal[1, i]
    
print(f'Be: {Be}')

epse = Be @ ae
print(f'eps: {epse}')

D = E / (1 - nu**2) * np.array([
    [1,     nu,     0           ],
    [nu,    1,      0           ],
    [0,     0,      (1 - nu) / 2]
])

sigmae = D @ epse
print(f'sigma [Mpa]: \n{sigmae * 10**(-6)}')


b)
ae: [[4.1e-05]
 [1.0e-05]
 [3.5e-05]
 [1.6e-05]
 [3.8e-05]
 [1.7e-05]]
ue: [[3.83e-05]
 [1.55e-05]]
Be: [[-130.43478261    0.           57.97101449    0.           72.46376812
     0.        ]
 [   0.          -86.95652174    0.          -72.46376812    0.
   159.42028986]
 [ -86.95652174 -130.43478261  -72.46376812   57.97101449  159.42028986
    72.46376812]]
eps: [[-0.00056522]
 [ 0.00068116]
 [ 0.00081159]]
sigma [Mpa]: 
[[-83.27759197]
 [118.06020067]
 [ 65.55183946]]


In [25]:
print('\nc)')

def b(x, y): 
    
    bx = 2 * x
    by = 3 * x * y
    b = np.array([
        [bx], 
        [by]
    ])
    return b

xi_val = 1/3 
eta_val = 1/3

x = Ne_bar_func(xi_val, eta_val) @ xe
y = Ne_bar_func(xi_val, eta_val) @ ye
print(f'x, y: {x, y}')

b_val = b(x[0][0], y[0][0])
print(f'b: {b_val}')
w = 1 / 2

Ne_val = Ne_func(xi_val, eta_val)

J_val = J_func(xi_val, eta_val)

fle = w * t * np.linalg.det(J_val) * Ne_val.T @ b_val
print(f'fl: {fle}')


c)
x, y: (array([[0.01533333]]), array([[0.01366667]]))
b: [[0.03066667]
 [0.00062867]]
fl: [[5.29000e-09]
 [1.08445e-10]
 [5.29000e-09]
 [1.08445e-10]
 [5.29000e-09]
 [1.08445e-10]]
